In [10]:
import numpy as np
import pandas as pd



In [11]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [12]:
!pip install gensim

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [45]:
df = pd.read_csv('Film_IBMDATA_set', nrows=50000)  # Adjust the number of rows as needed
df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


First step is converting all text into lower casing bold text since python is case sensative

In [32]:
df['review'][3].lower()

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.<br /><br />ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

In [33]:
df.shape

(50000, 2)

**To convert all review into the lower cases.**

In [34]:
df['review'].str.lower()

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. <br /><br />the...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    rififi, directed by jules dassin, is in line w...
49996    for anyone who's judged others at first meetin...
49997    i couldnt believe how well this kid did on scr...
49998    ok, i have been a huge fan of the black for a ...
49999    lorenzo lamas stars as some type of cia agent,...
Name: review, Length: 50000, dtype: object

In [35]:
df['review']=df['review'].str.lower()

In [36]:
df['review']

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. <br /><br />the...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    rififi, directed by jules dassin, is in line w...
49996    for anyone who's judged others at first meetin...
49997    i couldnt believe how well this kid did on scr...
49998    ok, i have been a huge fan of the black for a ...
49999    lorenzo lamas stars as some type of cia agent,...
Name: review, Length: 50000, dtype: object

**Remove unimportant information(HTML TAG) use this function to rem0ve html**

In [41]:
import re

def html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub('', text)



In [46]:
df['review_without_html'] = df['review'].apply(html_tags)
print(df[['review', 'review_without_html']].head())

                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                 review_without_html  
0  One of the other reviewers has mentioned that ...  
1  A wonderful little production. The filming tec...  
2  I thought this was a wonderful way to spend ti...  
3  Basically there's a family where a little boy ...  
4  Petter Mattei's "Love in the Time of Money" is...  


**Remove URLS for example whatsapp data or chat or wikipedia text**

In [47]:
import re

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)
text_with_urls = "Check out my website at https://www.example.com. For more information, visit http://anotherexample.com."
text_without_urls = remove_urls(text_with_urls)

print("Original text:", text_with_urls)
print("Text without URLs:", text_without_urls)


Original text: Check out my website at https://www.example.com. For more information, visit http://anotherexample.com.
Text without URLs: Check out my website at  For more information, visit 


**Remove Punctuations**

In [54]:
import string
import time


punctuation_chars = string.punctuation
print(punctuation_chars)


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [55]:
exclude=string.punctuation

In [56]:
import re
import string

def punctuation_chars(text, exclude=None):
    if exclude is None:
        exclude = string.punctuation

    for char in exclude:
        text = text.replace(char, '')

    return text

# Example usage:
text_with_punctuation = "Hello, world! This is an example text with punctuation."
text_without_punctuation = punctuation_chars(text_with_punctuation)

print("Original text:", text_with_punctuation)
print("Text without punctuation:", text_without_punctuation)


Original text: Hello, world! This is an example text with punctuation.
Text without punctuation: Hello world This is an example text with punctuation


In [65]:
start=time.time()
print(punctuation_chars)
time1=time.time()-start
print(time1)

<function punctuation_chars at 0x798787a5dd80>
0.00015306472778320312


**This is too slow for the fast processing use following functions to remove function.....**

In [60]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))


In [64]:
start=time.time()
print(punctuation_chars)
time2=time.time()-start
print(time2)

<function punctuation_chars at 0x798787a5dd80>
0.0017235279083251953


In [67]:
df['review'].apply(remove_punctuation)

0        One of the other reviewers has mentioned that ...
1        A wonderful little production br br The filmin...
2        I thought this was a wonderful way to spend ti...
3        Basically theres a family where a little boy J...
4        Petter Matteis Love in the Time of Money is a ...
                               ...                        
49995    Rififi directed by Jules Dassin is in line wit...
49996    For anyone whos judged others at first meeting...
49997    I couldnt believe how well this kid did on scr...
49998    OK I have been a huge fan of the Black for a l...
49999    Lorenzo Lamas stars as some type of CIA agent ...
Name: review, Length: 50000, dtype: object

**Chat word treatment or Salang word treatment** for this you have to used dictionary........... Use following function

In [68]:
def chat_conversation(text):
    new_text = []
    chat_words = {"R": "are", "U": "you", "L8": "late", "BRB": "be right back"}

    for word in text.split():
        if word.upper() in chat_words:
            new_text.append(chat_words[word.upper()])
        else:
            new_text.append(word)

    return ' '.join(new_text)

# Example usage:
input_text = "R U coming or not? BRB, I'm L8."
output_text = chat_conversation(input_text)

print("Original text:", input_text)
print("Converted text:", output_text)




Original text: R U coming or not? BRB, I'm L8.
Converted text: are you coming or not? BRB, I'm L8.


**Spelling correction for text preprocessing.It handel the common types of general mistake....**

In [72]:
from textblob import TextBlob
text_with_mistakes = "Thee quick brown fox jumpd ovver the lazy dog."
blob = TextBlob(text_with_mistakes)
corrected_text = blob.correct()

print("Original Text:", text_with_mistakes)
print("Corrected Text:", corrected_text)


Original Text: Thee quick brown fox jumpd ovver the lazy dog.
Corrected Text: Thee quick brown fox jumped over the lazy dog.


**Remove the stop words**

In [77]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stop_words)


{'yourself', 'him', 'do', 'under', 'those', 'don', 'mightn', "mustn't", 'during', 'hers', 'her', 'should', 'won', 'why', 'haven', 'am', 'what', 'these', 'wouldn', "she's", 'hasn', 'against', "it's", "you'd", 'own', "weren't", 'aren', "aren't", 'about', 'our', 'this', 'does', 'no', 'just', 'them', 'are', 've', 'above', 'doing', 'd', 'themselves', "won't", 'his', 'they', 'my', 'we', 'how', 's', 'myself', 'same', 'between', 'very', 'below', 'll', 'shan', 'the', 'hadn', 'weren', 'isn', 'having', 'a', 'into', 'further', 'y', "wasn't", 'being', 'up', "didn't", 'until', 'other', 'that', 'out', 'itself', 'needn', 'on', 'shouldn', 'then', 'down', 'more', 'at', 'where', 'have', 'and', 'than', 'of', 'when', 'himself', 're', "needn't", 'in', 'doesn', 'while', 'nor', "you're", "that'll", 'ourselves', 'with', "you'll", 'was', 'were', 'be', 'over', 'herself', 'ours', "mightn't", "shan't", "hasn't", "hadn't", 'mustn', 'yours', 'which', 'both', "wouldn't", 'its', 'there', "haven't", 'can', 'their', 'to

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# This function remove the stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)

    return filtered_text

# Example usage:
input_text = "This is an example sentence with some stopwords."
output_text = remove_stopwords(input_text)

print("Original Text:", input_text)
print("Text without Stopwords:", output_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Original Text: This is an example sentence with some stopwords.
Text without Stopwords: example sentence stopwords .


**Emoji Handeling Function.....** use the demojize function.....

In [86]:
!pip install emoji


In [88]:
import emoji

def remove_emojis(text):
    cleaned_text = emoji.demojize(text)
    cleaned_text = cleaned_text.replace(":", "")
    return cleaned_text

# Example usage:
text_with_emojis = "Hello! 😊 How are you today? 🌟"
text_without_emojis = remove_emojis(text_with_emojis)

print("Original Text:", text_with_emojis)
print("Text without Emojis:", text_without_emojis)


Original Text: Hello! 😊 How are you today? 🌟
Text without Emojis: Hello! smiling_face_with_smiling_eyes How are you today? glowing_star


**Tokanization very important step of NLP**
why tokanization is so important? It is helps to filter the number of Uniques words. while doing Tokenization alot challenges happen. for example $20 , New-york.....
Tokenization faces challenges such as handling word boundary ambiguity, deciding punctuation inclusion, and addressing complexities in abbreviations and acronyms. Multilingual variations, noisy text, sentence boundary identification, and customization for specific tasks like information extraction further contribute to the nuanced nature of tokenization challenges.

In [89]:
#1 use split function
def tokenize_with_split(text):
    # Tokenize using split function
    tokens = text.split()
    return tokens

# Example usage:
example_text = "Tokenization using split function is a basic approach."
tokens = tokenize_with_split(example_text)

print("Original Text:", example_text)
print("Tokens using split:", tokens)


Original Text: Tokenization using split function is a basic approach.
Tokens using split: ['Tokenization', 'using', 'split', 'function', 'is', 'a', 'basic', 'approach.']


**Handalin in tokenization**
 If you're experiencing issues with using the split function in Natural Language Processing (NLP), it's important to understand that the basic split function in Python is limited and may not handle certain tokenization challenges effectively, such as punctuation, contractions, or different languages.
 In NLP, more advanced tokenization methods are often preferred. The nltk library provides a more powerful word_tokenize function that can handle various tokenization challenges  **NLTK**

In [91]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenize_with_nltk(text):
    # Tokenize using nltk's word_tokenize
    tokens = word_tokenize(text)
    return tokens
# Example usage:
example_text = "Advanced NLP tokenization requires specialized tools?But I am going to new-york."
tokens_nltk = tokenize_with_nltk(example_text)

print("Original Text:", example_text)
print("Tokens using nltk:", tokens_nltk)


Original Text: Advanced NLP tokenization requires specialized tools?But I am going to new-york.
Tokens using nltk: ['Advanced', 'NLP', 'tokenization', 'requires', 'specialized', 'tools', '?', 'But', 'I', 'am', 'going', 'to', 'new-york', '.']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Spacy**
Spacy is a popular library for natural language processing (NLP) in Python, and it provides robust tools for tokenization and various other NLP tasks. Here's an example of how to use Spacy for tokenization:

In [94]:
import spacy


nlp = spacy.load("en_core_web_sm")
def tokenize_with_spacy(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]

    return tokens

# Example usage:
example_text = "I am Ph.D student."
tokens_spacy = tokenize_with_spacy(example_text)

print("Original Text:", example_text)
print("Tokens using Spacy:", tokens_spacy)


Original Text: I am Ph.D student.
Tokens using Spacy: ['I', 'am', 'Ph', '.', 'D', 'student', '.']


**Stemming**
Stemming is a text normalization process in natural language processing that involves reducing words to their root or base form. It helps in simplifying words to their common base, which can be useful for tasks like information retrieval and text analysis. Here's an example using the NLTK library for stemming:

In [98]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
porter_stemmer = PorterStemmer()

def stem_text(text):
    words = word_tokenize(text)
    stemmed_words = [porter_stemmer.stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

# Example usage:
original_text = "I am very interested in the NPL algorithm and i want to be very successful data scientist and ML engineer."
stemmed_text = stem_text(original_text)

print("Original Text:", original_text)
print("Stemmed Text:", stemmed_text)


Original Text: I am very interested in the NPL algorithm and i want to be very successful data scientist and ML engineer.
Stemmed Text: i am veri interest in the npl algorithm and i want to be veri success data scientist and ml engin .


**Snow ball** The Snowball Stemmer is another stemming algorithm, specifically designed to be more aggressive and language-specific than the Porter Stemmer. Here's an example using the Snowball Stemmer in NLTK:

In [99]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
# Initialize the Snowball Stemmer for English
snowball_stemmer = SnowballStemmer('english')
def snowball_stem_text(text):
    words = word_tokenize(text)
    stemmed_words = [snowball_stemmer.stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

# Example usage:
original_text = "Stemming with Snowball is more aggressive and language-specific."
stemmed_text_snowball = snowball_stem_text(original_text)

print("Original Text:", original_text)
print("Snowball Stemmed Text:", stemmed_text_snowball)


Original Text: Stemming with Snowball is more aggressive and language-specific.
Snowball Stemmed Text: stem with snowbal is more aggress and language-specif .


**Lemmatization** Lemmatization is a text normalization process in natural language processing that involves reducing words to their base or dictionary form (lemma). Unlike stemming, lemmatization ensures that the resulting word is a valid word by considering its context and part of speech. Here's an example using the NLTK library for lemmatization:

In [106]:
import nltk

# Download the WordNet resource
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_text = ' '.join(lemmatized_words)

    return lemmatized_text

# Example usage:
original_text = "I am going to University and talk with my professor for my ph.d research."
lemmatized_text = lemmatize_text(original_text)

print("Original Text:", original_text)
print("Lemmatized Text:", lemmatized_text)


Original Text: I am going to University and talk with my professor for my ph.d research.
Lemmatized Text: I am going to University and talk with my professor for my ph.d research .


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
